### Load needed libraries and make needed directories

In [ ]:
import scenicplus
scenicplus.__version__

import pycisTopic
pycisTopic.__version__

import os
import pandas as pd
import numpy as np
import pickle
import dill
import scanpy as sc
import sys
import warnings
import pyranges as pr
import seaborn as sns
from scipy.io import mmread
from scipy.sparse import csr_matrix
from pycisTopic.cistopic_class import *
from pycisTopic.lda_models import *
from pycisTopic.topic_binarization import *
from pycisTopic.diff_features import *
from pycistarget.utils import region_names_to_coordinates
from scenicplus.wrappers.run_pycistarget import run_pycistarget
from scenicplus.scenicplus_class import create_SCENICPLUS_object
from scenicplus.wrappers.run_scenicplus import run_scenicplus
from scenicplus.preprocessing.filtering import apply_std_filtering_to_eRegulons
from scenicplus.eregulon_enrichment import score_eRegulons
from scenicplus.cistromes import TF_cistrome_correlation, generate_pseudobulks

warnings.filterwarnings("ignore")
_stderr = sys.stderr
null = open(os.devnull,'wb')

projDir = os.getcwd()

## Define rankings, score and motif annotation database
# From https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/screen/mc_v10_clust/region_based/
# and https://resources.aertslab.org/cistarget/motif2tf/
db_anno_fpath = os.path.join(projDir, "input", "motif_db")
rankings_db = os.path.join(db_anno_fpath, 'hg38_screen_v10_clust.regions_vs_motifs.rankings.feather')
scores_db =  os.path.join(db_anno_fpath, 'hg38_screen_v10_clust.regions_vs_motifs.scores.feather')
motif_annotation = os.path.join(db_anno_fpath, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')

# From http://humantfs.ccbr.utoronto.ca/download/v_1.01/
tfdb = os.path.join(profDir, "input", "TF_names_v_1.01.txt")
biomart_host = "http://sep2019.archive.ensembl.org/"

# Output directories
outDir = os.path.join(pwd, "output")
if not os.path.exists(outDir):
    os.makedirs(outDir)

workDir = os.path.join(outDir, "tmp")
if not os.path.exists(workDir):
    os.makedirs(workDir)

if not os.path.exists(os.path.join(workDir, 'models')):
    os.makedirs(os.path.join(workDir, 'models'))

if not os.path.exists(os.path.join(workDir, 'candidate_enhancers')):
    os.makedirs(os.path.join(workDir, 'candidate_enhancers'))

if not os.path.exists(os.path.join(workDir, 'motifs')):
    os.makedirs(os.path.join(workDir, 'motifs'))

if not os.path.exists(os.path.join(workDir, 'scenicplus')):
    os.makedirs(os.path.join(workDir, 'scenicplus'))

### Read in needed datasets for cisTopic

In [ ]:
# Read the Matrix Market format file
data_folder = projDir
sparse_matrix = mmread(os.path.join(outDir, "sparse_matrix.mtx"))

# Convert to sparse.csr_matrix
sparse_csr_matrix = csr_matrix(sparse_matrix)

import pandas as pd
df = pd.read_csv(os.path.join(outDir, "/seqnames_ranges.csv"))

region_names = df.loc[:, 'Seqnames_Ranges'].tolist()
df = pd.read_csv(os.path.join(outDir, "/cell_names.csv"))
cell_names = df["cells"].tolist()

### Create cisTopic object and run

In [ ]:
cistopic_obj = create_cistopic_object(
    fragment_matrix=sparse_csr_matrix,
    cell_names=cell_names,
    region_names=region_names
)

## Checkpoint
pickle.dump(cistopic_obj, open(os.path.join(workDir, 'cistopic_obj.pkl'), 'wb'))

models = run_cgs_models(
    cistopic_obj,
    n_topics=[2,4,10,16,32,48],
    n_cpu=32,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    save_path=None,
)

pickle.dump(models, open(os.path.join(workDir, "models", "ATAC_models_500_iter_LDA.pkl"), 'wb'))

model = evaluate_models(
    models,
    select_model=16,
    return_model=True,
    metrics=['Arun_2010','Cao_Juan_2009', 'Minmo_2011', 'loglikelihood'],
    plot_metrics=False
)

cistopic_obj.add_LDA_model(model)

## Checkpoint
pickle.dump(cistopic_obj, open(os.path.join(workDir, 'cistopic_obj.pkl'), 'wb'))

region_bin_topics_otsu = binarize_topics(cistopic_obj, method='otsu')
region_bin_topics_top3k = binarize_topics(cistopic_obj, method='ntop', ntop = 3000)

cell_metadata = pd.read_csv(os.path.join(outDir, "cell_metadata.csv"), index_col=0)

cell_metadata = cell_metadata.reset_index()
cell_metadata["index"] = cell_metadata["index"] + "___cisTopic"

tmp = cistopic_obj.cell_data.copy()
tmp = tmp.reset_index()
tmp = pd.merge(tmp, cell_metadata, how='left', on='index')

tmp = tmp.set_index("index")
tmp.index.name = None

cistopic_obj.cell_data = tmp

imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

normalized_imputed_acc_obj = normalize_scores(
    imputed_acc_obj,
    scale_factor=10**4
)

variable_regions = find_highly_variable_features(
    normalized_imputed_acc_obj,
    plot=False
)

markers_dict = find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable='Subclass',
    var_features=variable_regions,
    split_pattern='-'
)

## Checkpoint
pickle.dump(cistopic_obj, open(os.path.join(workDir, 'cistopic_obj.pkl'), 'wb'))
pickle.dump(region_bin_topics_otsu, open(os.path.join(workDir, "candidate_enhancers", "region_bin_topics_otsu.pkl"), 'wb'))
pickle.dump(region_bin_topics_top3k, open(os.path.join(workDir, "candidate_enhancers", "region_bin_topics_top3k.pkl"), 'wb'))
pickle.dump(markers_dict, open(os.path.join(workDir, "candidate_enhancers", "markers_dict.pkl"), 'wb'))


region_sets = {}
region_sets['topics_otsu'] = {}
region_sets['topics_top_3'] = {}
region_sets['DARs'] = {}
for topic in region_bin_topics_otsu.keys():
    regions = region_bin_topics_otsu[topic].index[region_bin_topics_otsu[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['topics_otsu'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for topic in region_bin_topics_top3k.keys():
    regions = region_bin_topics_top3k[topic].index[region_bin_topics_top3k[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['topics_top_3'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for DAR in markers_dict.keys():
    regions = markers_dict[DAR].index[markers_dict[DAR].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['DARs'][DAR] = pr.PyRanges(region_names_to_coordinates(regions))


run_pycistarget(
    region_sets=region_sets,
    species='homo_sapiens',
    save_path=os.path.join(workDir, 'motifs'),
    ctx_db_path=rankings_db,
    dem_db_path=scores_db,
    path_to_motif_annotations=motif_annotation,
    run_without_promoters=True,
    n_cpu=30,
    annotation_version='v10nr_clust',
)

## Checkpoint
menr = dill.load(open(os.path.join(workDir, 'motifs/menr.pkl'), 'rb'))

### Read in needed datasets for scenicplus

In [ ]:
adata = sc.read_h5ad(os.path.join(projDir, "input", "SEAAD_MTG_RNAseq_final-nuclei.2024-02-13.h5ad")
adata = adata[(adata.obs["Class"] == "Non-neuronal and Non-neural") & (adata.obs["Severely Affected Donor"] == "N"), :].copy()
adata.X = adata.layers['UMIs'].copy()
del adata.layers['UMIs']

cistopic_obj = dill.load(open(os.path.join(workDir, "scATAC", "cistopic_obj.pkl"), 'rb'))
menr = dill.load(open(os.path.join(workDir, "motifs", "menr.pkl"), 'rb'))
adata.obs["Subclass"] = adata.obs['Subclass'].astype(str)

### Create scenicplus object and run

In [ ]:
scplus_obj = create_SCENICPLUS_object(
        GEX_anndata=adata,
        cisTopic_obj=cistopic_obj,
        menr=menr,
        multi_ome_mode=False,
        key_to_group_by="Subclass",
        nr_cells_per_metacells=5
)

# Checkpoint
dill.dump(scplus_obj, open(os.path.join(workDir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)

scplus_obj.dr_cell['eRegulons_UMAP'] = True


sys.stderr = open(os.devnull, "w")  # silence stderr
run_scenicplus(
    scplus_obj=scplus_obj,
    variable=['Subclass'],
    species="hsapiens",
    assembly="hg38",
    tf_file=tfdb,
    save_path=os.path.join(workDir, "scenicplus"),
    biomart_host=biomart_host,
    upstream=[1000, 150000],
    downstream=[1000, 150000],
    calculate_TF_eGRN_correlation=True,
    calculate_DEGs_DARs=True,
    export_to_loom_file=False,
    export_to_UCSC_file=False,
    n_cpu=30,
)
sys.stderr = sys.__stderr__  # unsilence stderr

### Filter eRegulons to produce final output

In [ ]:
scplus_obj = dill.load(open(os.path.join(workDir, 'scenicplus/scplus_obj.pkl'), 'rb'))
region_ranking = dill.load(open(os.path.join(workDir, 'scenicplus/region_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
gene_ranking = dill.load(open(os.path.join(workDir, 'scenicplus/gene_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
score_eRegulons(
    scplus_obj,
    ranking=region_ranking,
    eRegulon_signatures_key='eRegulon_signatures_filtered',
    key_added='eRegulon_AUC_filtered',
    enrichment_type='region',
    auc_threshold=0.05,
    normalize=False,
    n_cpu=30
)
score_eRegulons(
    scplus_obj,
    gene_ranking,
    eRegulon_signatures_key='eRegulon_signatures_filtered',
    key_added='eRegulon_AUC_filtered',
    enrichment_type='gene',
    auc_threshold=0.05,
    normalize=False,
    n_cpu=30
)

generate_pseudobulks(
    scplus_obj=scplus_obj,
    variable='label_transfer',
    auc_key='eRegulon_AUC_filtered',
    signature_key='Gene_based')
generate_pseudobulks(
    scplus_obj=scplus_obj,
    variable='label_transfer',
    auc_key='eRegulon_AUC_filtered',
    signature_key='Region_based'
)

TF_cistrome_correlation(
    scplus_obj,
    use_pseudobulk=True,
    variable='label_transfer',
    auc_key='eRegulon_AUC_filtered',
    signature_key='Gene_based',
    out_key='filtered_gene_based'
)
TF_cistrome_correlation(
    scplus_obj,
    use_pseudobulk=True,
    variable='label_transfer',
    auc_key='eRegulon_AUC_filtered',
    signature_key='Region_based',
    out_key='filtered_region_based'
)

n_targets = [int(x.split('(')[1].replace('r)', '')) for x in scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Cistrome']]
rho = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'].to_list()
adj_pval = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Adjusted_p-value'].to_list()

thresholds = {
        'rho': [-0.75, 0.70],
        'n_targets': 0
}

fig, ax = plt.subplots(figsize = (10, 5))
sc = ax.scatter(rho, n_targets, c = -np.log10(adj_pval), s = 5)
ax.set_xlabel('Correlation coefficient')
ax.set_ylabel('nr. target regions')
#ax.hlines(y = thresholds['n_targets'], xmin = min(rho), xmax = max(rho), color = 'black', ls = 'dashed', lw = 1)
ax.vlines(x = thresholds['rho'], ymin = 0, ymax = max(n_targets), color = 'black', ls = 'dashed', lw = 1)
ax.text(x = thresholds['rho'][0], y = max(n_targets), s = str(thresholds['rho'][0]))
ax.text(x = thresholds['rho'][1], y = max(n_targets), s = str(thresholds['rho'][1]))
sns.despine(ax = ax)
fig.colorbar(sc, label = '-log10(adjusted_pvalue)', ax = ax)
plt.show()

selected_cistromes = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].loc[
        np.logical_or(
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] > thresholds['rho'][1],
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] < thresholds['rho'][0]
        )]['Cistrome'].to_list()
selected_eRegulons = [x.split('_(')[0] for x in selected_cistromes]
selected_eRegulons_gene_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Gene_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]
selected_eRegulons_region_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Region_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]

#save the results in the scenicplus object
scplus_obj.uns['selected_eRegulon'] = {'Gene_based': selected_eRegulons_gene_sig, 'Region_based': selected_eRegulons_region_sig}
print(f'selected: {len(selected_eRegulons_gene_sig)} eRegulons')

# Checkpoint the selected results
dill.dump(scplus_obj, open(os.path.join(workDir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)

scplus_obj.uns['eRegulon_metadata_filtered'].to_csv("eRegulon_metadata_filtered.csv")